#### Pour que l'apprentissage par ML soit faisable, on sélectionne les **catégories significatives** avec un "grand" nombre d'exposés des faits associés.

## La démarche proposée

- on a créé ***déjà*** des modèles Doc2Vec abstraits
- on leur a associé comme vocabulaire les exposés déjà labélisés (avec comme tags les catégories présentes déjà dans la base de données et certains exposés peuvent apparaître plusieurs fois avec une même catégorie ou avec des catégories différentes)
- on a entraîné ces modèles avec comme documents les ***exposés des faits ayant comme tags les noms des catégories associées dans la base de données*** => on a obtenu autant de vecteurs numériques que de catégories distinctes
- ***maintenant***, avec les modèles Doc2Vec (entrainés), on infère les vecteurs numériques associés aux exposés des faits à classifier (on réalise l'embedding) pour tous les exposés et on garde comme classes (labels) pour la suite les "bonnes" catégories (mentionnées dans la base de données)
- avec tous ces vecteurs vus comme des vecteurs de features, on fait la classification supervisée grâce à des modèles LogisticRegression

In [1]:
import random
import pickle

import spacy
nlp = spacy.load('fr')
spacy.load('en')

import numpy as np
import pandas as pd

from textblob import TextBlob as tb

from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

## Chargement des modèles Doc2Vec entraînés et sauvés

Les 2 modèles ci-dessous correspondent à l'embedding (à la vectorisation) des edf à l'aide de la librairie Doc2Vec :
- **model_dbow_300** utilise l'algorithme DBOW, une taille de vecteurs de 300 features et a été entraîné 30 epochs
- **model_dm_300** utilise l'algorithme DM, une taille de vecteurs de 300 features et a été entraîné 30 epochs

In [113]:
model_dbow_300 = Doc2Vec.load("model_dbow_300_61.model")

In [114]:
model_dm_300 = Doc2Vec.load("model_dm_300_61.model")

**l_main_cats** est une liste avec les catégories principales, i.e. avec plus de 1000 exposés associés

In [6]:
with open('l_main_cats_61', 'rb') as f:
    l_main_cats = pickle.load(f)
#print(len(l_main_cats))
#print(l_main_cats[:3])
#print(l_main_cats)

**ll_merged_cats** est la liste de liste qui regroupe les catégories principales semblables

In [5]:
with open('ll_merged_cats_61', 'rb') as f:
    ll_merged_cats = pickle.load(f)
print(len(ll_merged_cats))
print(ll_merged_cats)

8
[['ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 'AR - DIVERS', 'INFRACTION - LCR', 'AR - PANNE - VEHICULE', 'CIRCULATION - ROUTIERE', 'AR - ACCIDENT', 'ACCIDENT - DE CIRCULATION - AVEC FUITE', 'ACCIDENT - DE CIRCULATION - AVEC BLESSE', 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL', 'VEHICULE - SUSPECT', 'VEHICULE'], ['TAPAGE NOCTURNE', 'TROUBLE - DE LA TRANQUILLITE / NUISANCE', 'BRUIT'], ['VOL - PAR EFFRACTION', 'VOL', "VOL - A L'ETALAGE"], ['INDIVIDU - SUSPECT', 'INDIVIDU - PERTURBE', "FUITE - D'UN LIEU DE PLACEMENT"], ['OPERATION', 'REVOCATION', 'APPREHENSION / ARRESTATION', 'COLLABORATION - INTERPOLICE', 'DEMANDE - IDENTIFICATION'], ['BAGARRE', 'LITIGE', 'VIOLENCE - DOMESTIQUE'], ["DEMANDE - D'AMBULANCE", "DEMANDE - D'ASSISTANCE"], ['ANIMAUX', 'INCENDIE', 'DROGUE', 'MINEUR - IMPLIQUE', 'TENTATIVE', 'DOMMAGES - A LA PROPRIETE', 'INDESIRABLE']]


**l_merged_catNoms** est une liste avec les noms des catégories principales regroupées

In [7]:
with open('l_merged_catNoms_61', 'rb') as f:
    l_merged_catNoms = pickle.load(f)
print(len(l_merged_catNoms))
print(l_merged_catNoms)

8
['CIRCULATION', 'TROUBLE', 'VOLS', 'INDIVIDU', 'INTERVENTION', 'DISPUTE', 'DEMANDE', 'DIVERS']


**main_efs** est le DataFrame contenant les exposés (et leurs catégories) associés aux catégories "principales" (avec plus de 1000 edf correspondants)

In [11]:
main_efs = pd.read_csv('/home/alina/UNIFR/TM/random/ExposesDesFaits/main_efs_61.csv', header=0, sep=';')
#main_efs.head(5)

**merged_main_efs** est la base de données dans laquelle on a ajouté la colonne supplémentaire pour le regroupement des catégories principales

In [36]:
merged_main_efs = pd.read_csv('/home/alina/UNIFR/TM/random/ExposesDesFaits/merged_main_efs_61.csv', header=0, sep=';')
merged_main_efs.head(5)

,EXPOSES,CATEGORIE,MERGED_CAT
0,Un renard sur la voie gauche.,ANIMAUX,DIVERS
1,Bras cassé suite glissade sur le verglas.,DEMANDE - D'AMBULANCE,DEMANDE
2,Demande d'ambulance pour un jeune qui a trop b...,DEMANDE - D'AMBULANCE,DEMANDE
3,"Un véhicule noire, plaques AI commençant évent...",ACCIDENT - DE CIRCULATION - AVEC FUITE,CIRCULATION
4,panne moteur,AR - PANNE - VEHICULE,CIRCULATION


In [43]:
print('Dans la base de données, il y a', merged_main_efs.shape[0], 'enregistrements.')

Dans la base de données, il y a 122204 enregistrements.


## Logistic Regression

Par la suite, on fait une classification supervisée grâce à la logistic regression. 

L'ensemble de données de départ est formé par : 
- les vecteurs numériques (feature vectors) obtenus par l'embedding des exposés des faits à l'aide du modèle Doc2Vec
- les tags assoiés aux feature vectors (en fait les catégories)

### Préparation des données (d'entraînement et de test) pour le modèle LR

**l_expos** est la liste des exposés des faits dont la catégorie associée est une des catégories "principales" (avec plus de 1000 edf correspondants)

In [12]:
l_expos = list(merged_main_efs['EXPOSES'])
l_expos[:3]

['Un renard sur la voie gauche.',
 'Bras cassé suite glissade sur le verglas.',
 "Demande d'ambulance pour un jeune qui a trop bu et qui ne se réveille pas. Appel transféré au 144."]

**l_cats** est la liste des catégories associées aux exposés des faits de la liste **l_expos** (dans le même ordre)

In [13]:
l_cats = list(merged_main_efs['MERGED_CAT'])
l_cats[:3]

['DIVERS', 'DEMANDE', 'DEMANDE']

**l_couples_expo_cat** est la liste des couples dont le premier élément est un exposé de faits et le deuxième élément est la catégorie qui lui est associée (dans la base de donnée princiapale **main_efs**)

In [14]:
l_couples_expo_cat = list(zip(l_expos, l_cats))
l_couples_expo_cat[:3]

[('Un renard sur la voie gauche.', 'DIVERS'),
 ('Bras cassé suite glissade sur le verglas.', 'DEMANDE'),
 ("Demande d'ambulance pour un jeune qui a trop bu et qui ne se réveille pas. Appel transféré au 144.",
  'DEMANDE')]

**l_couples_expo_cat_rand** est obtenu par la randomization de la liste **l_couples_expo_cat**

In [15]:
l_couples_expo_cat_rand = random.sample(l_couples_expo_cat, len(l_couples_expo_cat))
print ("La liste originale : \n",  l_couples_expo_cat[:3])
print ("\nLa liste après la randomization non in place : \n",  l_couples_expo_cat_rand[:3])

La liste originale : 
 [('Un renard sur la voie gauche.', 'DIVERS'), ('Bras cassé suite glissade sur le verglas.', 'DEMANDE'), ("Demande d'ambulance pour un jeune qui a trop bu et qui ne se réveille pas. Appel transféré au 144.", 'DEMANDE')]

La liste après la randomization non in place : 
 [('une biche morte dans le champ ', 'DIVERS'), ('Dame très âgée désorientée.', 'DEMANDE'), ("Affaire provenant de réception HP: L'informatrice avait laissé sa valise chez sa soeur. Cette dernière n'habite plus l'appartement et à laissé sa valise sur place. Le nouveau locataire refuse de lui rendre sa valise. Va sur place et nous rappelle.", 'DEMANDE')]


On sépare (graĉe à la fonction **train_test_split**) l'ensemble des couples **l_couples_expo_cat_rand** en 2 sous-ensembles : un pour l'entraînement (90%) avec le suffixe **train** et l'autre pour le test (10%) avec le suffixe **test**

In [16]:
l_couples_expo_cat_train, l_couples_expo_cat_test = \
            train_test_split(l_couples_expo_cat_rand, test_size=0.1, random_state=2000)

In [45]:
print('Les', len(l_couples_expo_cat_rand), 'enregistrements ont été divisés en', 
      len(l_couples_expo_cat_train), 'enregistrements pour l\'entraînement et', 
      len(l_couples_expo_cat_test), 'enregistrements pour le test.')

Les 122204 enregistrements ont été divisés en 109983 enregistrements pour l'entraînement et 12221 enregistrements pour le test.


On extrait les exposés pour l'entraînement et pour le test.

In [17]:
l_expos_train = [expo for expo, cat in l_couples_expo_cat_train]
l_expos_test = [expo for expo, cat in l_couples_expo_cat_test]

On extrait les catégories pour l'entraînement et pour le test.

In [18]:
l_cats_train = [cat for expo, cat in l_couples_expo_cat_train]
l_cats_test = [cat for expo, cat in l_couples_expo_cat_test]

### L'embedding en vue de la classification LR

Grâce à l'un des modèles Doc2Vec déjà entraînés, on peut faire l'embedding, i.e. calculer le vecteur numérique (feature vector), pour n'importe quel exposé des faits (qui fait partie ou pas de l'ensemble d'entraînement du modèle Doc2Vec)

La fonction **edf2vec** reçoit comme paramètres:
- **expo** l'exposé des faits à vectoriser
- **model_d2v** le modèle Doc2Vec qui va inférer le vecteur numérique (feature vector)
- **epochs** le nombre d'itérations durant l'entraînement du nouveau document
- **seed** le "seed" pour la génération de nombres aléatoires pour le model Doc2Vec utilisé 

et elle retourne le nouveau feature vector dont la dimension est celle prévue par le modèle qui infère.

Remarque : des appels successifs de la méthode prédéfinie **infer_vector** peuvent retourner des vecteurs numériques différents.

In [19]:
def edf2vec(expo, model_d2v, epochs=None, seed=8017):
    if epochs is None:
        epochs=model_d2v.epochs
    blob = tb(expo.lower())
    doc_words = list(blob.words)
    spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS
    # on élimine les stop words
    doc_words = [elem for elem in doc_words if elem not in spacy_stop_words]
    # on élimine les mots contenant au moins un chiffre
    doc_words = [elem for elem in doc_words if not any(c.isdigit() for c in elem)]
    # on élimine les mots formés d'un seul caractère
    doc_words = [elem for elem in doc_words if len(elem) > 1]
    #print(doc_words)
    model_d2v.random.seed(seed)
    v = model_d2v.infer_vector(doc_words, epochs)
    return v

## Création des modèles LogisticRegression abstrait

Pour chaque modèle Doc2Vec, on crée un modèle abstrait de type LogisticRegression (et les 2 modèles abstraits sont identiques)

In [115]:
model_lr_dbow_300 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

In [116]:
model_lr_dm_300 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

## Concrétisation des modèles abstraits et évaluation de leurs performances

Il faut associer au modèle abstrait les données d'entraînement (training data), à savoir :
- l'ensemble de vecteurs d'entraînement (training vectors) représenté par une matrice (tableau bidimensionnel) ayant comme lignes les feature vectors calculés pour les exposés des faits grâce au modèle (entraîné) Doc2Vec
- le vecteur avec les labels (target vector) représenté par la liste **l_labels** contenant les tags pour les exposés des faits (dans le même ordre que celui de l'ensemble de vecteurs d'entraînement)


On fait l'embedding (la vectorisation), donc on calcule (on infère) les vecteurs numériques pour l'**entraînement** et pour le **test** pour chacun des 2 modèles Doc2Vec (grâce à la méthode ad-hoc edf2vec qui fait appel à la méthode standard infer_vector)

In [117]:
%%time
vects_train_dbow_300 = [edf2vec(expo, model_dbow_300) for expo in l_expos_train]
vects_test_dbow_300 = [edf2vec(expo, model_dbow_300) for expo in l_expos_test]

CPU times: user 1min 5s, sys: 203 ms, total: 1min 5s
Wall time: 1min 5s


In [118]:
vects_train_dbow_300[0].shape

(500,)

In [119]:
%%time
vects_train_dm_300 = [edf2vec(expo, model_dm_300) for expo in l_expos_train]
vects_test_dm_300 = [edf2vec(expo, model_dm_300) for expo in l_expos_test]

CPU times: user 1min 19s, sys: 183 ms, total: 1min 20s
Wall time: 1min 20s


In [120]:
vects_train_dm_300[0].shape

(500,)

**On entraîne les 2 modèles LR abstraits** (identiques) avec les vecteurs numériques et les labels d'entraînement (suffixe **train**) correspondants à chaque modèle Doc2Vec 

L'entraînement se fait grâce à la méthode **fit** et suite à son appel, le modèle abstrait appelant la méthode devient un modèle entraîné

Par la suite, le modèle entraîné sera capable de faire des prédiction pour les catégories (les classes) grâce aux méthodes **predict()** et **predict_proba()**

In [121]:
%%time
model_lr_dbow_300.fit(vects_train_dbow_300, l_cats_train)

CPU times: user 279 ms, sys: 1.5 s, total: 1.78 s
Wall time: 7min 24s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=-1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [122]:
%%time
model_lr_dm_300.fit(vects_train_dm_300, l_cats_train)

CPU times: user 364 ms, sys: 749 ms, total: 1.11 s
Wall time: 7min 30s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=-1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

Avec chacun des modèles LR entraînés, **on prédit les catégories** (les classes ou les labels) pour les vecteurs numériques de test

En fait, appelée avec une matrice dont les lignes sont les feature vectors à classifier, la méthode **predict** retourne un array  avec autant d'éléments que de lignes dans la matrice et dont chaque élément représente la classe (chez nous la catégorie) la plus probable pour le feature vector correspondant

In [123]:
l_cats_pred_dbow_300 = model_lr_dbow_300.predict(vects_test_dbow_300)

In [124]:
l_cats_pred_dm_300 = model_lr_dm_300.predict(vects_test_dm_300)

**On calcule les performances** de chacun des 2 modèles LR grâce à des fonctions prévues à cet effet dans la librairie ***sklearn.metrics***

In [125]:
print('L\'accuracy pour le modèle dbow_300 :', accuracy_score(l_cats_test, l_cats_pred_dbow_300))
print('Le score F1 pour le modèle dbow_300 :', f1_score(l_cats_test, l_cats_pred_dbow_300, average='weighted'))
print('La precision pour le modèle dbow_300 :', precision_score(l_cats_test, l_cats_pred_dbow_300, average='weighted'))
print('Le recall pour le modèle dbow_300 :', recall_score(l_cats_test, l_cats_pred_dbow_300, average='weighted'))

L'accuracy pour le modèle dbow_300 : 0.6974879306112429
Le score F1 pour le modèle dbow_300 : 0.6935074807805897
La precision pour le modèle dbow_300 : 0.6925520945398996
Le recall pour le modèle dbow_300 : 0.6974879306112429


In [126]:
print('L\'accuracy pour le modèle dm_300 :', accuracy_score(l_cats_test, l_cats_pred_dm_300))
print('Le score F1 pour le modèle dm_300 :', f1_score(l_cats_test, l_cats_pred_dm_300, average='weighted'))
print('La precision pour le modèle dm_300 :', precision_score(l_cats_test, l_cats_pred_dm_300, average='weighted'))
print('Le recall pour le modèle dm_300 :', recall_score(l_cats_test, l_cats_pred_dm_300, average='weighted'))

L'accuracy pour le modèle dm_300 : 0.5619834710743802
Le score F1 pour le modèle dm_300 : 0.5410531131402272
La precision pour le modèle dm_300 : 0.5599675228795
Le recall pour le modèle dm_300 : 0.5619834710743802


On peut vérifier les valeurs obtenues pour l'accuracy en appelant la méthode **score** pour chaque modèle LR déjà entraînée (à la place de la fonction **accuracy_score** du package sklearn.metrics)

In [127]:
print('L\'accuracy pour le modèle dbow_300 :', model_lr_dbow_300.score(vects_test_dbow_300, l_cats_test))

L'accuracy pour le modèle dbow_300 : 0.6974879306112429


In [128]:
print('L\'accuracy pour le modèle dm_300 :', model_lr_dm_300.score(vects_test_dm_300, l_cats_test))

L'accuracy pour le modèle dm_300 : 0.5619834710743802


De plus, on peut aussi calculer la matrice de confusion qui est une matrice carrée dont les lignes correspondent aux catégories réelles (de la base de données) et les colonnes correspondent aux catégories prédites (par le modèle LR).

In [129]:
conf_matrix_test_dbow_300 = confusion_matrix(l_cats_test, l_cats_pred_dbow_300)
print(conf_matrix_test_dbow_300)

[[3242   86    8  104   43   39   12   45]
 [ 160 1146  113  201  132   35   68   97]
 [  18  150  345   51   16   13   19   27]
 [ 177  166   31 1186  106   87   38  170]
 [  66  213   27  102  509  106   28   82]
 [  80   59   23   59   94  651    2   81]
 [  18   38   13   45    8    8  756   24]
 [  67   66    6   87   26   42   15  689]]


In [130]:
l_tot_lines = np.sum(conf_matrix_test_dbow_300, axis=1)
print(l_tot_lines)

[3579 1952  639 1961 1133 1049  910  998]


In [131]:
l_tot_lines.sum()

12221

In [132]:
conf_matrix_test_dm_300 = confusion_matrix(l_cats_test, l_cats_pred_dm_300)
print(conf_matrix_test_dm_300)

[[3166  151    1  134   27   33   20   47]
 [ 488  946   48  234   74   66   62   34]
 [ 142  186  168   67   12   19   32   13]
 [ 598  232   13  828   54   79   53  104]
 [ 254  259    6  160  294   82   29   49]
 [ 333  118    7   85   53  405    4   44]
 [ 105   71    5   89    4    7  617   12]
 [ 241   77    5  135   27   47   22  444]]


La matrice de confusion est une matrice carrée dont les lignes correspondent aux catégories réelles et les colonnes correspondent aux catégories prédites.

Dans la matrice de confusion:
- Sur la diagonale principale, on trouve tous les TP (true positive)
    - par exemple, dans la cellule d'indices i=j=5, se trouve le nombre d'edf qui ont été prédits comme appartenant à la catégorie 5 et qui appartenaient en réalité aussi à la catégorie 5
- Sur une ligne, toutes les cellules à part celle sur la diagonale principale contiennent des F (false) qui peuvent être interprêtés comme des FN (false negative)
    - par exemple, dans la celule d'indices i=3 et j=2, se trouve le nombre d'edf qui appartiennent en réalité à la catégorie 3, mais qui ont été prédits comme appartenant à la catégorie 2
- Sur une colonne, toutes les cellules à part celle sur la diagonale principale contiennent des F (false) qui peuvent être interprêtés comme des FP (false positive)
    - par exemple, dans la celule d'indices i=3 et j=2, se trouve le nombre d'edf qui appartiennent en réalité à la catégorie 3, mais qui ont été prédits comme appartenant à la catégorie 2
    
De plus:
- la somme de toutes les cellules situées sur une même ligne donne le nombre d'edf qui en réalité ont comme type la catégorie correspondante à cette ligne
    - par exemple, la somme de toutes les cellules situées sur la ligne 3 correspond au nombre d'edf qui appartiennent en réalité à la catégorie 3
- la somme de toutes les cellules situées sur une même colonne donne le nombre d'edf qui ont été prédits comme appartenant à la catégorie correspondante à cette colonne
    - par exemple, la somme de toutes les cellules situées sur la colonne 2 correspond au nombre d'edf qui ont été prédits comme appartenant à la catégorie 2

L'ordre dans lequel les catégories sont prises en compte par le modèle LogisticRegression

In [ ]:
print(model_lr_dbow_300.classes_)

In [ ]:
print('La première ligne \n', conf_matrix_test_dbow_300[0,])
print('\n La première colonne \n', conf_matrix_test_dbow_300[:,0])

Interprétation de la première ligne :
- (cellule 0,0) 153 edf ont dans la base de données la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ont été prédites correctement
- (cellule 0,3) 4 edf ont dans la base de données la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ont été prédites comme 'ACCIDENT - DE CIRCULATION - DEGATS MATERIEL'
- (cellule 0,4) 8 edf ont dans la base de données la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ont été prédites comme 'ANIMAUX'

Interprétation de la première colonne :
- (cellule 0,0) 153 edf ont été prédits dans la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ils ont dans la base de données la même catégorie
- (cellule 3,0) 4 edf ont été prédits dans la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ils ont dans la base de données la catégorie 'ACCIDENT - DE CIRCULATION - DEGATS MATERIEL'
- (cellule 4,0) 13 edf ont été prédits dans la catégorie 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL' et ils ont dans la base de données la catégorie 'ANIMAUX'

### Remarque importante
- l'étude de la première ligne et de la première colonne de la matrice de confusion met en évidence que les catégories **'ACCIDENT - DE CIRCULATION - AVEC ANIMAL'** et les catégories **'ACCIDENT - DE CIRCULATION - DEGATS MATERIEL'** et **'ANIMAUX'** produisent des confusions (sont difficilement distinguables et un même exposé pourrait être mis à juste titre dans 2 catégories)

In [133]:
vects_train_dbow_dm = [np.array(list(vects_train_dbow_300[i])+list(vects_train_dm_300[i])) 
                       for i in range(len(vects_train_dbow_300))]

In [134]:
vects_test_dbow_dm = [np.array(list(vects_test_dbow_300[i])+list(vects_test_dm_300[i])) 
                       for i in range(len(vects_test_dbow_300))]

In [135]:
vects_train_dbow_dm[0].shape

(1000,)

In [136]:
model_lr_dbow_dm = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

In [137]:
%%time
model_lr_dbow_dm.fit(vects_train_dbow_dm, l_cats_train)

CPU times: user 576 ms, sys: 6.28 s, total: 6.86 s
Wall time: 14min 36s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=-1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [138]:
l_cats_pred_dbow_dm = model_lr_dbow_dm.predict(vects_test_dbow_dm)

In [139]:
print('L\'accuracy pour le modèle dbow_dm :', accuracy_score(l_cats_test, l_cats_pred_dbow_dm))

L'accuracy pour le modèle dbow_dm : 0.7220358399476311


In [140]:
print('L\'accuracy pour le modèle dbow_dm :', model_lr_dbow_dm.score(vects_test_dbow_dm, l_cats_test))

L'accuracy pour le modèle dbow_dm : 0.7220358399476311


## Prédiction pour un nouvel exposé des faits

Pour prédire la "bonne" catégorie pour un nouvel exposé des faits :
- d'abord, on crée un modèle abstrait LogisticRegression et on l'entraîne avec des vecteurs numériques (obtenus grâce à un modèle Doc2Vec) et des labels correspondants
- ensuite, on calcule le vecteur numérique (feature vector) correspondant au nouvel exposé grâce à l'embedding (ou la vectorisation) réalisé(e) avec le (même) modèle Doc2Vec
- finalement, on prédit la "bonne" catégorie grâce au modèle LogisticRegression

### Création et entraînement d'un modèle logistique

On crée de nouveaux modèles abstraits LR avec le suffixe **fin** (basés ensuite sur les modèles Doc2Vec)

In [ ]:
model_lr_dbow_300_fin = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

In [ ]:
model_lr_dm_300_fin = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

Afin d'entraîner ces modèles abstraits LR, on utilise comme ensemble de vecteurs numériques d'entraînement les vecteurs numériques inférés par les modèles Doc2Vec pour ***tous*** les exposés

In [ ]:
%%time
X_dbow_300_fin = [edf2vec(expo, model_dbow_300) for expo,cat in l_couples_expo_cat_rand]

In [ ]:
%%time
X_dm_300_fin = [edf2vec(expo, model_dm_300) for expo,cat in l_couples_expo_cat_rand]

**y_fin** est le vecteur cible (target vector) contenant les catégories (labels) correspondants à l'ensemble d'entraînement et ce vecteur est le même pour tous les 2 modèles

In [ ]:
%%time
y_fin = [cat for expo,cat in l_couples_expo_cat_rand]

On entraîne les modèles abstraits sur l'ensemble d'entraînement

In [ ]:
%%time
model_lr_dbow_300_fin.fit(X_dbow_300_fin,y_fin)

In [ ]:
%%time
model_lr_dm_300_fin.fit(X_dm_300_fin,y_fin)

### Embedding d'un nouvel exposé des faits
Grâce au modèle Doc2Vec déjà entraîné, on peut calculer le vecteur numérique (feature vector) d'un nouvel exposé des faits qui n'est pas dans l'ensemble d'entraînement grâce à la méthode ad-hoc **edf2vec** définie auparavant (et qui fait appel à la méthode **infer_vector** du modèle Doc2Vec)

**edf_new** est un nouvel edf (qui n'est pas dans l'ensemble d'entraînement)

In [ ]:
edf_new = 'Un véhicule est en panne sur la route d\'Oron. Il bloque la voie de droite de la route.'
#edf_new = 'Les voisins font beaucoup trop de bruit. La musique est trop forte.'

**v_new** est le vecteur numérique inféré pour le nouvel exposé

In [ ]:
v_new_dbow_300 = edf2vec(edf_new, model_dbow_300)
#print(edf_new, '\n')
#print('Le vector inféré pour un document nouveau vaut:\n\n', v_new)

In [ ]:
v_new_dm_300 = edf2vec(edf_new, model_dm_300)

### Prédiction de la "bonne" catégorie

On peut obtenir la catégorie proposée (prédite) par le modèle LR pour le vecteur numérique correspondant au nouvel exposé **edf_new** grâce à la méthode **predict**

En fait, la méthode **predict** retourne la catégorie (la classe) pour laquelle la probabilité de correspondre au nouvel exposé des faits est la plus grande

In [ ]:
cat_proposed_dbow_300 = model_lr_dbow_300_fin.predict([v_new_dbow_300])
print('La catégorie proposée est pour dbow_300:', cat_proposed_dbow_300)
#print(type(cat_new))

In [ ]:
cat_proposed_dm_300 = model_lr_dm_300_fin.predict([v_new_dm_300])
print('La catégorie proposée est pour dm_300:', cat_proposed_dm_300)

On peut maintenant regrouper ces opérations en une seule fonction définie ci-dessous.

La fonction **infer_cat** permet de déterminer la "bonne" catégorie pour un exposé donné et elle :
- a comme paramètres :
    - **expo** l'exposé des faits à classifier 
    - **model_d2v** le modèle Doc2Vec pour l'embedding (la vectorisation)
    - **model_lr** le modèle LogisticRegression pour la classifiction multinomiale (prédiction de la catégorie)
    - **epochs** le nombre d'époques (d'itérations) pour l'inférence du vecteur numérique    
    - **seed** le "seed" pour la génération de nombres aléatoires pour le model Doc2Vec utilisé 
- affiche et retourne la "bonne" catégorie

In [ ]:
def infer_cat(expo, model_d2v, model_lr, epochs=None, seed=8017):
    print('L\'edf :', expo)
    if epochs is None:
        epochs=model_d2v.epochs
    # on vectorise l'edf (on infère le vecteur numérique avec le modèle Doc2vec)
    vect = edf2vec(expo, model_d2v, epochs, seed)
    # on calcule (prédit) la "bonne" catégorie (avec le modèle LogisticRegression)
    cat_proposed = model_lr.predict([vect])
    print('La catégorie proposée est :', cat_proposed)
    return cat_proposed

Des exemples d'appel de la fonction **infer_cat** pour un même exposé des faits mais pour des modèles différents

In [ ]:
infer_cat(edf_new, model_dbow_300, model_lr_dbow_300_fin)

In [ ]:
infer_cat(edf_new, model_dm_300, model_lr_dm_300_fin)

### Prédiction des "meilleures" catégories

Vu qu'assez souvent un exposé des faits n'a pas forcément une unique catégorie qui peut lui correspondre, il est judicieux de faire **plusieurs propositions** de catégories (en fonction de leurs probabilités de similarité) et pour cela on va utiliser la méthode **predict_proba**.

Appelée avec une matrice dont les lignes sont les feature vectors à classifier, la méthode **predict_proba** retourne aussi une matrice (un array bidimensionnel) avec le même nombre de lignes que la matrice argument et un nombre de colonnes égal au nombre de classes (chez nous des catégories) distinctes du modèle.

Plus précisément, la ligne i de la matrice retournée contient les probabilités par classes correspondantes aux vecteurs numériques i de la matrice argument => la somme des éléments d'une ligne de la matrice retournée doit être égale à 1.

Les classes (les catégories) distinctes sont ordonnées dans le modèle comme dans l'attribut **classes_**.

Pour un seul vecteur numérique donné en argument, la méthode **predict_proba** permet d'obtenir les probabilités qu'il soit classifié dans chacune des catégories prises en compte (et la somme de toutes ces probabilités doit valoir 1).

Plus préciséement, le résultat retourné par la méthode **predict_proba** est une matrice (tableau bidimensionnel) avec une seule ligne qui contient, dans un certain ordre, les probabilités de classification du nouvel exposé des faits dans chacune des catégories (classes) considérées 

Afin de connaître l'ordre des catégories auxquelles correspondent les probabilités retournées par la méthode **predict_proba**, on utilise l'attribut **classes_** pour le modèle LogisticRegression employé.

On explique d'abord les étapes à suivre pour le cas ***dbow_300*** et on regroupe ensuite la démarche prposée dans une fonction ad-hoc nommée **show_infer_cats**

In [ ]:
edf_new = 'Un véhiule est en panne sur la route d\'Oron. Il bloque la voie de droite de la route.'
#edf_new = 'Les voisins font beaucoup trop de bruit. La musique est trop forte.'

In [ ]:
v_new = edf2vec(edf_new, model_dbow_300)

In [ ]:
ll_cats_new_proba = model_lr_dbow_300_fin.predict_proba([v_new])
print(ll_cats_new_proba[0][:5])

In [ ]:
#print(type(model_lr_dbow_300_fin.classes_))
l_classes = list(model_lr_dbow_300_fin.classes_)
print(l_classes[:5])

On veut maintenant ordonner de manière ***décroissante*** les probabilités retournées par la méthode **predict_proba** afin de trouver et afficher les ***meilleures*** catégories à proposer.

In [ ]:
l_couples = list(enumerate(ll_cats_new_proba[0]))
print(l_couples[:5])

In [ ]:
l_couple_sorted = sorted(l_couples, key=lambda tup:tup[1], reverse=True)
print(l_couple_sorted[:5])

On affiche les meilleures propositions de catégories pour le nouvel exposé des faits

In [ ]:
print(edf_new, '\n************************')
print('Les catégories recommandées sont: \n')
for i in range(5):
    no_cat = l_couple_sorted[i][0]
    cat = l_classes[no_cat]
    print(cat, ' (avec la probabilité de ', l_couple_sorted[i][1]*100, '%)')
    print('---------------------------')

On peut maintenant regrouper les opérations précédantes en une seule fonction définie ci-dessous.

La fonction **show_infer_cats** permet de déterminer les "meilleures" catégories pour un exposé donné et elle :
- a comme paramètres :
    - **expo** l'exposé des faits à classifier 
    - **model_d2v** le modèle Doc2Vec pour l'embedding (la vectorisation)
    - **epochs** le nombre d'époques (d'itérations) pour l'inférence du vecteur numérique
    - **model_lr** le modèle LogisticRegression pour la classifiction multinomiale (prédiction de la catégorie)
    - **nb** le nombre de "meilleures" catégories
    - **seed** le "seed" pour la génération de nombres aléatoires pour le model Doc2Vec utilisé 
- affiche les meilleures catégories

In [ ]:
def show_infer_cats(expo, model_d2v, model_lr, nb=5, epochs=None, seed=8017):
    if epochs is None:
        epochs=model_d2v.epochs
    # on vectorise l'edf (on infère le vecteur numérique avec le modèle Doc2vec)       
    vect = edf2vec(expo, model_d2v, epochs, seed)
    # on calcule (prédit) le probabilités de TOUTES les catégories (avec le modèle LogisticRegression)
    ll_cats_proba = model_lr.predict_proba([vect])
    #print(ll_cats_proba)
    # on associe les probabilités avec l'indice de leur apparition dans la liste précédante
    l_couples = list(enumerate(ll_cats_proba[0]))
    # on ordonne les probabilités de manière décroissante (en gardant leur indice initial)
    l_couple_sorted = sorted(l_couples, key=lambda tup:tup[1], reverse=True)
    #print(l_couple_sorted)
    # on affiche les nb meilleures catégories avec leurs probabilités
    print(expo, '\n********************************************************')
    print('Les catégories recommandées sont: \n')
    for i in range(nb):
        no_cat = l_couple_sorted[i][0]
        cat = model_lr.classes_[no_cat]
        print(cat, ' (avec la probabilité de ', l_couple_sorted[i][1]*100, '%)')
        print('------------------------------------------')

### Exemples de prédictions des meilleures catégories

Ci-dessous, on présente des exemples de "meilleures" catégories prédites pour de nouveaux exposés des faits (par de simples appels de la méthode **show_infer_cats**) pour les 2 modèles LR considérés

### Exemple 1

In [ ]:
edf_new = 'Un véhicule est en panne sur la route d\'Oron. Il bloque la voie de droite de la route.'

In [ ]:
show_infer_cats(edf_new, model_dbow_300, model_lr_dbow_300_fin)

In [ ]:
show_infer_cats(edf_new, model_dm_300, model_lr_dm_300_fin)

### Exemple 2

In [ ]:
edf_new = 'Les voisins font beaucoup trop de bruit. La musique est trop forte.'

In [ ]:
show_infer_cats(edf_new, model_dbow_300, model_lr_dbow_300_fin)

In [ ]:
show_infer_cats(edf_new, model_dm_300, model_lr_dm_300_fin)